Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.919585
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.758892
Minibatch accuracy: 56.2%
Validation accuracy: 60.2%
Minibatch loss at step 100: 0.514387
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 150: 1.141887
Minibatch accuracy: 81.2%
Validation accuracy: 75.9%
Minibatch loss at step 200: 0.572379
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 250: 0.465786
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.555426
Minibatch accuracy: 75.0%
Validation accuracy: 78.7%
Minibatch loss at step 350: 0.542803
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.495972
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.294891
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.133913
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In the model above the first and second layers are both convolutions with stride [1,2,2,1] ([1, stride, stride, 1]).


In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    print(data)
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print(conv)
    print(hidden)
    print(pool)
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print(conv)
    print(hidden)
    print(pool)
    
    shape = pool.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Tensor("Placeholder:0", shape=(16, 28, 28, 1), dtype=float32)
Tensor("Conv2D:0", shape=(16, 28, 28, 16), dtype=float32)
Tensor("Relu:0", shape=(16, 28, 28, 16), dtype=float32)
Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)
Tensor("Conv2D_1:0", shape=(16, 14, 14, 16), dtype=float32)
Tensor("Relu_1:0", shape=(16, 14, 14, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(16, 7, 7, 16), dtype=float32)
[16, 7, 7, 16]
Tensor("Const:0", shape=(10000, 28, 28, 1), dtype=float32)
Tensor("Conv2D_2:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("Relu_3:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Conv2D_3:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("Relu_4:0", shape=(10000, 14, 14, 16), dtype=float32)
Tensor("MaxPool_3:0", shape=(10000, 7, 7, 16), dtype=float32)
[10000, 7, 7, 16]
Tensor("Const_1:0", shape=(10000, 28, 28, 1), dtype=float32)
Tensor("Conv2D_4:0", shape=(10000, 28, 28, 16), dtype=float32

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.064257
Minibatch accuracy: 6.2%
Validation accuracy: 9.2%
Minibatch loss at step 50: 2.032565
Minibatch accuracy: 43.8%
Validation accuracy: 40.9%
Minibatch loss at step 100: 0.772150
Minibatch accuracy: 68.8%
Validation accuracy: 69.5%
Minibatch loss at step 150: 1.412769
Minibatch accuracy: 68.8%
Validation accuracy: 61.9%
Minibatch loss at step 200: 0.515858
Minibatch accuracy: 81.2%
Validation accuracy: 76.7%
Minibatch loss at step 250: 0.485844
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.703733
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.658951
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 400: 0.456594
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.197555
Minibatch accuracy: 93.8%
Validation accuracy: 82.1%
Minibatch loss at step 500: 0.133020
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The LeNet architecture is actually the same as above model we have.
Input - Conv1 - Relu1 - Pool1 - Conv2 - Relu2 - Pool2 - Densely Connected Relu3 - Drop out - Read out (Output)

Tried to tune the batch_size, num_steps, num_hidden, depth, dropout, learning rate decay. 

In [70]:
batch_size = 16
patch_size = 5
depth1 = 16
depth2 = 16
num_hidden = 196

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  # layer3 is the densely connected layer
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob=1):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Shape [batch, row, column, depth]
    print("Conv_1 shape: ", conv.get_shape().as_list())
    print("Relu_1 shape: ", hidden.get_shape().as_list())
    print("Pool_1 shape: ", pool.get_shape().as_list())
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print("Conv_2 shape: ", conv.get_shape().as_list())
    print("Relu_2 shape: ", hidden.get_shape().as_list())
    print("Pool_2 shape: ", pool.get_shape().as_list())
          
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print("KeepProb: ", keep_prob)
    hidden_drop = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden_drop, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 1)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           200, 0.9, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Conv_1 shape:  [16, 28, 28, 16]
Relu_1 shape:  [16, 28, 28, 16]
Pool_1 shape:  [16, 14, 14, 16]
Conv_2 shape:  [16, 14, 14, 16]
Relu_2 shape:  [16, 14, 14, 16]
Pool_2 shape:  [16, 7, 7, 16]
KeepProb:  1
Conv_1 shape:  [10000, 28, 28, 16]
Relu_1 shape:  [10000, 28, 28, 16]
Pool_1 shape:  [10000, 14, 14, 16]
Conv_2 shape:  [10000, 14, 14, 16]
Relu_2 shape:  [10000, 14, 14, 16]
Pool_2 shape:  [10000, 7, 7, 16]
KeepProb:  1
Conv_1 shape:  [10000, 28, 28, 16]
Relu_1 shape:  [10000, 28, 28, 16]
Pool_1 shape:  [10000, 14, 14, 16]
Conv_2 shape:  [10000, 14, 14, 16]
Relu_2 shape:  [10000, 14, 14, 16]
Pool_2 shape:  [10000, 7, 7, 16]
KeepProb:  1


In [71]:
num_steps = 3001

def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Do shuffle at the begining each time loop start from dataset
    if offset <= batch_size:
      train_dataset, train_labels = randomize(train_dataset, train_labels)    
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    if (step % 300 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Learning rate: %f' % learning_rate.eval())
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.474729
Learning rate: 0.100000
Minibatch accuracy: 0.0%
Validation accuracy: 11.6%
Minibatch loss at step 300: 0.721424
Learning rate: 0.090000
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 600: 0.721633
Learning rate: 0.072900
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 900: 0.169288
Learning rate: 0.065610
Minibatch accuracy: 93.8%
Validation accuracy: 85.7%
Minibatch loss at step 1200: 0.307647
Learning rate: 0.053144
Minibatch accuracy: 93.8%
Validation accuracy: 85.6%
Minibatch loss at step 1500: 0.385826
Learning rate: 0.047830
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 1800: 0.251285
Learning rate: 0.038742
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 2100: 0.283023
Learning rate: 0.034868
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 2400: 0.910743
Learning rate: 0.028243
Minibatch acc